# Assignment 1

Authors: 
- Andreas Rosenquist (s214604)
- Felix Lund Frandsen ()
- Kasper Rønberg (s214786)


## Assignment 1.1: Exploring WS and BA models

In [ ]:
# Import necessary libraries
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import random
import numpy as np
import requests
import os
import re

# Set seaborn style
sns.set(style="darkgrid")

### Part 1:

#### Exercises: Did you really read the text? Answer the following questions (no calculations needed) in your IPython notebook.

**Why Random Networks are better model for real-world Networks**

Random Networks will not create clear outliers. As an example, a random network could not replicate the properties of a social network, as there would be no highly popular person, or someone with zero friends (outliers). As described in the book; in a large random network the degree of most nodes would be close to the average degree $\langle k \rangle$ $^{(1)}$.

**The four Regimes of Random Networks:**

- Subcritical: $\langle k \rangle < 1$
- Critical Point: $\langle k \rangle = 1$  
- Supercritcal Regime: $\langle k \rangle > 1$
- Connected Regime: $\langle k \rangle > \ln N$ $^{(2)}$

**Problems of Clustering Coefficient $C(k)$ for random networks compared to real-world networks**

In Random Networks, $C(k)$ decreases with the increase of $N$. In other words, the cluserting coefficent decreases when the size of the random network increases. This is problematic as a model for real-world networks, because real-world networks and their clustering coefficient are independent of network size, $N$. $^{(3)}$


**References** 
(1) Network Science Book, Chapter 3.5
(2) Network Science Book, Chapter 3.6
(3) Network Science Book, Chapter 3.9

#### Exercises: WS edition.

In [ ]:
N = 500
k = 4
p = [0, 0.1, 1]

WG0 = nx.watts_strogatz_graph(N, k, p[0])
WG1 = nx.watts_strogatz_graph(N, k, p[1])
WG2 = nx.watts_strogatz_graph(N, k, p[2])  

In [ ]:
WG0_avg_path_length = nx.average_shortest_path_length(WG0)
WG1_avg_path_length = nx.average_shortest_path_length(WG1)
WG2_avg_path_length = nx.average_shortest_path_length(WG2)
print("Average path length for p=0: ", WG0_avg_path_length)
print("Average path length for p=0.1: ", WG1_avg_path_length)
print("Average path length for p=1: ", WG2_avg_path_length)

**Watt-Strogatz model at Rewiring Probability $(p)$ of 1**

Increasing the paramter p will increase the randomness of the network, i.e. reducing the clustering of the random network. An intermediate $p$ will resemble the small world phenomenon of random networks, as the wiring of the edges will be more random and not only focus on the k-nearest neighbours. At $p=1$ we see that the average shortest path length increase significantly from $p=0.1$, this would indicate a futher reduction in the clustering. In other words, when $p=1$, the network will fully neglect the characteristc of wiring to close neighbours, but instead  choose uniformly at random to wire an edge to any node in the network. This will naturally result in shorter paths as the network becomes more intertwined.

In [ ]:
extracted_ds = {}

ps = [0, 0.01, 0.03, 0.05, 0.1, 0.2]
N = 500
k = 4

for i in range(50):
    for p in ps:
        WSG = nx.watts_strogatz_graph(N, k, p)
        avg_path_length = nx.average_shortest_path_length(WSG)
        if p not in extracted_ds:
            extracted_ds[p] = [avg_path_length] 
        else:
            extracted_ds[p].append(avg_path_length)


In [ ]:
# code block brought to you by chatGPT
extracted_ds_avg = {p: sum(lengths)/len(lengths) for p, lengths in extracted_ds.items()}
extracted_ds_sd = {p: (sum((x - extracted_ds_avg[p]) ** 2 for x in lengths) / len(lengths))**0.5 for p, lengths in extracted_ds.items()}

In [ ]:
ds_avg = list(extracted_ds_avg.values())
ds_sd = list(extracted_ds_sd.values())

In [ ]:
plt.errorbar(ps, ds_avg, yerr=ds_sd, fmt='o', ecolor='r', capthick=2, color='black')
plt.plot(ps, ds_avg, linestyle='-', color='black')
plt.xlabel('Rewiring Probability (p)')
plt.ylabel('Average Shortest Path Length ($<d>$)')
plt.title('Average Shortest Path Length vs Rewiring Probability')
figure_text = """
Figure 1: Average Shortest Path Length, <d>, as a function of Rewiring Probability, p, in a Watts-Strogatz Graph. The figure highlights that an increase in rewiring probability p,
reduces clustering of the random network which in turn affects the average path lengths - reducing the average path length between nodes.
"""
plt.figtext(0.5, -0.15, 
            figure_text,
            wrap=True, 
            horizontalalignment='center', 
            fontsize=10
            )
plt.show()

### Part 2:

#### Exercises: BA edition.

**The three slope-dependent regimes of complex networks with power-law degree distributions**

- **Anomalous Regime ($\gamma \leq 2$):**  
  The number of links of the largest hub grows faster than the size of the network. For sufficiently large $N$, this brings forth anomalous features of scale-free networks, such as the largest hub exceeding the total number of nodes - essentially running out of nodes to connect to. In this regime, the average degree $\langle k \rangle$ diverges, allowing an infinite number of large hubs as $N \rightarrow \infty$ $^{(1)}$.  

- **Scale-Free Regime ($2 < \gamma < 3$):**  
  This regime is characterized by a finite first moment of the degree distribution, $\langle k \rangle$, and a divergent second moment, $\langle k^2 \rangle$ $^{(1)}$. As $N \rightarrow \infty$, the tail of the degree distribution increases, creating a network that resembles an ultra small-world network. In other words, hubs with large degrees will grow without bound:  
  $$
  \langle k^2 \rangle \rightarrow \infty \quad \text{as} \quad N \rightarrow \infty
  $$  

- **Random Network Regime ($\gamma > 3$):**  
  In this regime, both the first and second moments of the degree distribution are finite. The network tends toward the structure of a small-world network, and the properties of the scale-free network become difficult to distinguish from those of a random network of similar size. However, scale-free properties can still be observed, though doing so often requires unrealistically large network sizes $^{(1)}$.  

**The three regimes in non-linear preferential attachment**  

- **Sublinear Regime ($0 < \alpha < 1$):**  
  This preferential attachment generates minimal to no hubs, and the structure resembles that of a random network. However, the degree distribution follows a stretched exponential, where the largest hub has a degree approximated by $\ln t^{\frac{1}{1-\alpha}}$ at time $t$ $^{(2)}$.  

- **Linear Regime ($\alpha = 1$):**  
  Preferential attachment in the linear regime yields a network with scale-free properties, essentially mirroring the Barabási-Albert model. The degree distribution follows a power law, and the degree of the largest hub at time $t$ is given by $t^{\frac{1}{\gamma - 1}}$, where $\gamma$ is the degree exponent of the scale-free network $^{(2)}$.  

- **Superlinear Regime ($\alpha > 1$):**  
  In this regime, the tendency to link new nodes to already large nodes is amplified. This preferential attachment can therefore be described as a “rich-get-richer” process. The degree distribution is heavy-tailed with extremely large hubs, where the highest-degree node grows with time $t$ $^{(2)}$.  

\* The tendency mentioned here refers to a property of the Barabási-Albert model: earlier nodes tend to grow larger than later-arriving nodes. In other words, nodes generated early in the construction of the network have a “first-mover advantage,” as their opportunities to attract new links are much higher than those of later arrivals.  

**References**  
(1) *Network Science Book*, Chapter 4.7  
(2) *Network Science Book*, Chapter 5.8  


In [ ]:
# Initialize the BA graph with two connected nodes
BA = nx.Graph()
BA.add_node(1)
BA.add_node(2)
BA.add_edge(1, 2)

# Loop from 3 to 5000+1 to populate the graph with 5000 nodes
for i in range(3, 5000+1):
    # Add the new node
    BA.add_node(i)
    # We choose an existing node to connect to, with probability proportional to its degree (here we use itertools.chain to flatten the edge list)
    chosen_node = random.choice(list(itertools.chain.from_iterable(edge for edge in BA.edges)))
    # Add the edge between the new node and the chosen existing node
    BA.add_edge(i, chosen_node)


In [ ]:
# Plot BA graph
plt.figure(figsize=(12, 10))
pos = nx.spring_layout(BA, k=0.2, iterations=100, seed=420)
nx.draw(BA, pos=pos, node_size=7, width=0.5)
plt.title(label="Home-Brewed Barabasi-Albert Random Graph")
plt.show()

In [ ]:
# Gather edges for each node
edges_count = {}
for i in range(1, 5001):
    edges_count[i] = len(BA.edges(i))

In [ ]:
# code block brought to you by chatGPT
# Finds max and min degree nodes and their degrees
max_degree_node = max(edges_count, key=edges_count.get)
print("Node with highest degree:", max_degree_node)
print("Degree of that node:", edges_count[max_degree_node])

min_degree_node = min(edges_count, key=edges_count.get)
print("Node with lowest degree:", min_degree_node)
print("Degree of that node:", edges_count[min_degree_node])

In [ ]:
# Create bins and edges based on degree counts and bin ranges based on maximum degree
bins, edges = np.histogram(
                        list(edges_count.values()), 
                        bins=range(1,  edges_count[max_degree_node]+2)
                        )

In [ ]:
plt.figure(figsize=(10, 4))
plt.scatter(edges[:-1], bins, marker='v')
plt.ylabel('Frequency')
plt.xlabel('Degree (k)')
plt.title('Degree Distribution of Home-Brewed Barabasi-Albert Random Network')
plt.figtext(0.5, -0.10, 
            "Figure 2: Degree Distribution of a Home-Brewed Barabasi-Albert Random Network (linear scale). The distribution shows the frequency of nodes having a specific degree k.",
            wrap=True, 
            horizontalalignment='center', 
            fontsize=10
            )
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.scatter(edges[:-1], bins, marker='v')
plt.ylabel('Frequency')
plt.xlabel('Degree (k)')
plt.title('Degree Distribution of Home-Brewed Barabasi-Albert Random Network')
plt.yscale('log')
plt.xscale('log')
plt.figtext(0.5, -0.10, 
            "Figure 2: Degree Distribution of a Home-Brewed Barabasi-Albert Random Network (log scale). The distribution shows the frequency of nodes having a specific degree k.",
            wrap=True, 
            horizontalalignment='center', 
            fontsize=10
            )
plt.show()

## Assignment 1.2 Stats and visualization of the Rock Music Network

### 1.2.1 Explain your process in words

#### First, simply describe the steps of the process (what you did, step by step).

First, we collected a list of rock bands and performers from a Wikipedia page. Using regular expressions, we extracted all the links from this page. The performer names were then isolated, and the corresponding pages were downloaded, stored as JSON files, and saved to a text file. Next, we initialized a directed graph using networkx. Each performer was added as a node, with their word count stored as a node attribute. We then scraped the performer text file for links, and if a link matched another performer in our dataset, we added it as a directed edge. Finally, we removed isolated nodes and extracted the largest weakly connected component. 

#### Then, write a short section about which part of the process that was most difficult.

The most challenging part was extracting the JSON files from the Wikipedia pages, as we encountered a 403 Bad Gateway error. Furthermore, many wiki-links were outdated and required redirecting, which caused us to extract empty wiki-pages. This caused several issues and required a debugging session. We ultimately resolved the problems by specifying a user agent and adding the redirect as a parameter.

#### Next write a short section about how you used LLMs to help you.

We used LLMs to help debug the code when we encountered problems and to review our code for potential errors that could lead to incorrect data extraction.

#### Finally, compose a short section about what you would do differently if you had to do it again

We would set up stronger filtering before downloading pages; making sure to drop categories and resolve redirects before fetching the wiki-pages. We would also implement error handling from the start to catch any empty wiki-pages by having a minimum characters filter and an error warning for fetching bad files. 

### 1.2.3 Simple network statistics and analysis

##### Download and load the network from github.

In [ ]:
# download the graph as a gml file from github
url = "https://raw.githubusercontent.com/AIAndreas/SG-IAssignment1/main/data/rock_performers_graph.gml"
response = requests.get(url)

with open("rock_performers_graph.gml", "wb") as file:
    file.write(response.content)

# Load the graph from the gml file
G = nx.read_gml("rock_performers_graph.gml")

#### What is the number of nodes in the network?

In [ ]:
print(f"The graph has {G.number_of_nodes()} nodes.")

#### More importantly, what is the number of links?

In [ ]:
print(f"The graph has {G.number_of_edges()} edges.")

#### Plot the in and out-degree distributions for the entire network. What do you observe? Can you explain why the in-degree distribution is different from the out-degree distribution?

- Compare the out-degree distribution to a random network with the same number of nodes and links.

- Compare the in-degree distribution to a scale-free network with the same number of nodes.


In [ ]:
# Reference graphs for comparison
random_graph = nx.gnm_random_graph(
    G.number_of_nodes(), G.number_of_edges(), 
    directed=True
    )

scale_free_graph = nx.scale_free_graph(
    G.number_of_nodes()).to_directed()

in_degrees_G = [d for n, d in G.in_degree()]
out_degrees_G = [d for n, d in G.out_degree()]

out_degrees_random = [d for n, d in random_graph.out_degree()]
in_degrees_scale_free = [d for n, d in scale_free_graph.in_degree()]

def plot_degree_distribution(degrees, title, ax, xlabel='Degree (k)', ylabel='Frequency'):
    bins, edges = np.histogram(degrees, bins=range(1, max(degrees)+2))
    ax.scatter(edges[:-1], bins, marker='o', alpha=0.6)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xscale('log')
    ax.set_yscale('log')

fig, axs = plt.subplots(2, 2, figsize=(18, 10))
plot_degree_distribution(out_degrees_G, 'Out-Degree Distribution of Rock Performers Graph', axs[0, 0])
plot_degree_distribution(in_degrees_G, 'In-Degree Distribution of Rock Performers Graph', axs[0, 1])
plot_degree_distribution(out_degrees_random, 'Out-Degree Distribution of Random Graph', axs[1, 0])
plot_degree_distribution(in_degrees_scale_free, 'In-Degree Distribution of Scale-Free Graph', axs[1, 1])
plt.tight_layout()
plt.show()

The random network follows the expected Poisson distribution, with most nodes having degrees clustered around the mean and very few outliers. In contrast, the rock performers’ out-degree distribution is more dispersed and exhibits a heavier tail. This means that while most performers reference only a few others, some reference disproportionately many. 

The rock performers’ in-degree distribution shows a clear heavy-tailed pattern, even more pronounced than the scale-free network. This suggests a preferential attachment pattern (insert ref); performers who are already well-known or influential are disproportionately more likely to be referenced by new entrants, leading to the emergence of hubs.

#### Who are the top 5 most connected performers (Report results for in-degrees and out-degrees, that is, who has highest in-degree, who has highest out-degree)? Comment on your findings. Is this what you would have expected?

In [ ]:
# find top 5 most connected performers by in-degree and out-degree
top_in_degree = sorted(G.in_degree(), key=lambda x: x[1], 
                       reverse=True)[:5]

top_out_degree = sorted(G.out_degree(), key=lambda x: x[1], 
                        reverse=True)[:5]

# Remove text in parentheses and underscores from performer names
def normalize_label(raw_name: str) -> str:
    label = raw_name.replace('_', ' ')
    label = re.sub(r'\s*\(.*?\)\s*', '', label)
    return label.strip()

top_in_degree = [(normalize_label(name), degree) for name, degree in top_in_degree]
top_out_degree = [(normalize_label(name), degree) for name, degree in top_out_degree]

print("Top 5 performers by in-degree:")
for i, (performer, degree) in enumerate(top_in_degree):
    print(f"{i+1}. {performer}, In-Degree: {degree}")

print("\nTop 5 performers by out-degree:")
for i, (performer, degree) in enumerate(top_out_degree):
    print(f"{i+1}. {performer}, Out-Degree: {degree}")


The performers with the highest in-degrees are well-known rock bands or artists with significant influence and recognition in the music industry. This makes sense because their Wikipedia pages are frequently referenced by other performers, likely due to their historical importance, collaborations, or influence on the genre. 

On the other hand, the performers with the highest out-degrees include some lesser-known artists. This observation is also logical, as a high out-degree suggests that these performers' Wikipedia pages link to many other performers. This could be due to their extensive collaborations, inspirations, or influences from a wide range of artists.

#### What are the 10 pages with the longest wiki entries? (use the length of content attribute to figure this one out)?

In [ ]:
node_word_counts = [(n, d['word_count']) for n, d in G.nodes(data=True)]
top_10_longest = sorted(node_word_counts, key=lambda x: x[1], reverse=True)[:10]

print("Top 10 pages with the longest wiki entries:")
for i, (node, count) in enumerate(top_10_longest):
    print(f"{i+1}. {node}, Word Count: {count}")

## Assignment 3 Let's build a simple visualization of the network


#### For the sake of the visualisation, let's convert our network to undirected graph (tip: There is a NetworkX command to help you).



In [ ]:
G_undir = G.copy().to_undirected()

#### Use the NetworkX command nx.spring_layout or nx.draw_kamada_kawai to draw the resulting undirected network.

* Set up your plot so that node-size depends on the node degree.

* Make the node color depend on the length of content attribute. I recommend choosing a color scheme that is quite simple

In [ ]:
spring_pos = nx.spring_layout(G_undir, k=20, scale=20, seed=42)
kamada_pos = nx.kamada_kawai_layout(G_undir, pos=spring_pos, weight=None, scale=20)

deg = dict(G_undir.degree())
deg_vals = np.array([deg[n] for n in G_undir.nodes()])

# Get word counts directly from node attributes
words = [p['word_count'] for n, p in G_undir.nodes(data=True)]
sizes = (120 + 12 * deg_vals)*0.2 
cmap = plt.cm.inferno

plt.figure(figsize=(12, 9))
nx.draw_networkx_edges(G_undir, kamada_pos, alpha=0.15, width=0.4)
nodes = nx.draw_networkx_nodes(
    G_undir, kamada_pos,
    node_size=sizes,
    node_color=words,
    cmap=cmap,
    linewidths=0.25,
    edgecolors='black',
)

plt.title('Rock Performers Network')
colorbar = plt.colorbar(nodes)
colorbar.set_label('Words (page length)')
plt.axis('off')
plt.tight_layout()
plt.show()